In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from warnings import filterwarnings
import torch
import torchkbnufft as tkbn

In [2]:
#res in cm
res = 0.1
#fov in cm
fov_x = 5
fov_y = fov_x

pixels_x = int(fov_x/res)
pixels_y = int(fov_y/res)
pixels = pixels_x*pixels_y
print(pixels_x)
t1_myelin = 4.0e-1 
t1_wm = 8.5e-1      
t2_myelin = 5.0e-2  
t2_wm = 5.0e-2     
t2_star_myelin = 3.0e-2  
t2_star_wm = 7.0e-2    
inv_time = np.log(2)*t1_wm
echo_time = 4e-5

50


NameError: name 'np' is not defined

In [ ]:
image = np.ones((pixels_x, pixels_y))

mxy_myelin = (1-2*np.exp(-(inv_time/t1_myelin)))*np.exp(-(echo_time/t2_star_myelin))
mxy_wm = (1-2*np.exp(-(inv_time/t1_wm)))*np.exp(-(echo_time/t2_star_wm))
mxy_mixed = 0.5*((1-2*np.exp(-(inv_time/t1_wm)))*np.exp(-(echo_time/t2_star_wm))) + 0.5*((1-2*np.exp(-(inv_time/t1_myelin)))*np.exp(-(echo_time/t2_star_myelin)))

print(mxy_myelin)
print(mxy_wm)
print(mxy_mixed)

outer_square_size = int(1.0 * pixels_x) 
middle_square_size = int(0.6 * pixels_x)
inner_square_size = int(0.25 * pixels_x)  

# Calculate starting and ending indices for each square to center them
outer_start = (pixels_x - outer_square_size) // 2
outer_end = outer_start + outer_square_size

middle_start = (pixels_x - middle_square_size) // 2
middle_end = middle_start + middle_square_size

inner_start = (pixels_x - inner_square_size) // 2
inner_end = inner_start + inner_square_size

# Define the intensity values for each section
# Outermost square (100% WM - white, intensity = 1)
image[outer_start:outer_end, outer_start:outer_end] = mxy_wm  # 100% WM

# Middle square (50/50 of WM and myelin - medium gray)
image[middle_start:middle_end, middle_start:middle_end] = mxy_mixed  # 50/50 of WM and myelin (gray)

# Innermost square (100% myelin - black, intensity = 0)
image[inner_start:inner_end, inner_start:inner_end] = mxy_myelin  # 100% myelin (black)


# Display the image using matplotlib
plt.imshow(image, cmap='gray', interpolation='nearest')
plt.axis('off')  # Turn off axis
plt.colorbar()
plt.show()



In [ ]:
f_image = np.fft.fft2(image)

fft_image_shifted = np.fft.fftshift(f_image)  # Shift the zero-frequency component to the center

# Compute the magnitude spectrum
magnitude_spectrum = np.log(np.abs(fft_image_shifted) + 1)  # Log to enhance visibility

# Plot the original image and its FFT magnitude spectrum
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image, cmap='gray')
plt.colorbar()
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title("FFT Magnitude Spectrum - Log")
plt.imshow(magnitude_spectrum, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
filterwarnings("ignore") # ignore floor divide warnings
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
im_size = image.shape
grid_size = (tuple(int(dim * 2) for dim in im_size))
print(im_size,grid_size)

In [ ]:
# convert the phantom to a tensor and unsqueeze coil and batch dimension
image_t = torch.tensor(image).to(device).unsqueeze(0).unsqueeze(0)
print('image shape: {}'.format(image_t.shape))

In [ ]:
#res in mm and pixels
res_mm = 1
max_radii = 0.5/res_mm
dis_points = 1/pixels_x
print(max_radii)
nspokes = int(2*np.pi*max_radii*pixels_x) # Number of rays
num_samples_per_ray = int(max_radii*pixels_x) # Number of samples per ray
print(nspokes,num_samples_per_ray)
# Initialize an empty k-space list to store sampled points
k_space = np.zeros((nspokes * num_samples_per_ray,), dtype=complex)

# Create radial k-space sampling points (rays)
angles = np.linspace(0, 2 * np.pi, nspokes, endpoint=False)  # Angle of the rays
radii = np.linspace(0, max_radii, num_samples_per_ray)  # Radial distance (from the center)

# Loop over rays to calculate Fourier coefficients
kx_values = []
ky_values = []
for angle in angles:
    for radius in radii:
        # Calculate the kx, ky coordinates for each sampled point
        kx = radius * np.cos(angle)  # x-component in frequency domain
        ky = radius * np.sin(angle)  # y-component in frequency domain

        # Store kx and ky for density compensation
        kx_values.append(kx)
        ky_values.append(ky)

# Convert kx_values and ky_values to numpy arrays
kx_values = np.array(kx_values)
ky_values = np.array(ky_values)

ktraj = np.stack((ky_values.flatten(), kx_values.flatten()), axis=0)
print(ktraj.shape)

# Find min and max values across all axes
k_min = np.min(ktraj)
k_max = np.max(ktraj)

# Scale to [-pi, pi]
ktraj_scaled = (ktraj - k_min) / (k_max - k_min) * (2 * np.pi) - np.pi
ktraj = ktraj_scaled

# Visualize the radial k-space sampling points (this shows the rays)
plt.figure(figsize=(6, 6))
for angle in angles:
    x_vals = np.linspace(0, max_radii * np.cos(angle), num_samples_per_ray)
    y_vals = np.linspace(0, max_radii * np.sin(angle), num_samples_per_ray)
    plt.plot(x_vals, y_vals, 'b.', markersize=1)
plt.title('Radial k-space Trajectory')
plt.xlabel('kx')
plt.ylabel('ky')
plt.axis()
plt.grid(True)

# Adjust the axis limits to provide padding for text
padding = 0.5 * max_radii  # 10% of max_radii for padding
plt.xlim(-max_radii - padding, max_radii + padding)
plt.ylim(-max_radii - padding, max_radii + padding)

# Plot markers on the axes at kmax and -kmax
plt.plot([max_radii, -max_radii], [0, 0], 'ro', label='kx markers')  # Markers along x-axis
plt.plot([0, 0], [max_radii, -max_radii], 'ro', label='ky markers')  # Markers along y-axis

# Add text labels near the markers with offsets
text_offset = 0.05 * max_radii  # Offset for the text relative to the marker
plt.text(max_radii + text_offset, 0, 'k x,max', color='red', fontsize=10, ha='left', va='center')  # x-axis positive
plt.text(-max_radii - text_offset, 0, '-k x,max', color='red', fontsize=10, ha='right', va='center')  # x-axis negative
plt.text(0, max_radii + text_offset, 'k y,max', color='red', fontsize=10, ha='center', va='bottom')  # y-axis positive
plt.text(0, -max_radii - text_offset, '-k y,max', color='red', fontsize=10, ha='center', va='top')  # y-axis negative

plt.show()
#print(ktraj)


In [ ]:
# convert k-space trajectory to a tensor
ktraj = torch.tensor(ktraj).to(device)
print('ktraj shape: {}'.format(ktraj.shape))

In [ ]:
print(im_size,grid_size)

# create NUFFT objects, use 'ortho' for orthogonal FFTs
nufft_ob = tkbn.KbNufft(
    im_size=im_size,
    grid_size=grid_size,
).to(image_t)
adjnufft_ob = tkbn.KbNufftAdjoint(
    im_size=im_size,
    grid_size=grid_size,
).to(image_t)

print(nufft_ob)
print(adjnufft_ob)

In [ ]:
# plot the kernel
fig, axs = plt.subplots(1, 2)
axs.flat[0].plot(np.real(nufft_ob.table_0.cpu().numpy()))
axs.flat[1].plot(np.imag(nufft_ob.table_0.cpu().numpy()))
    
plt.show()

In [ ]:
#print(image_t.shape)

# Convert image_t to complex if it's real (with last dimension not equal to 2)
if image_t.ndimension() == 3 and image_t.shape[-1] != 2:  # Real input
    print("Converting real image to complex...")
    image_t = torch.stack([image_t, torch.zeros_like(image_t)], dim=-1)  # Convert to complex

#print(image_t.shape)

# Check the dtype of image_t
#print(image_t.dtype)

# If it's real, convert it to complex
if image_t.dtype != torch.complex128:
    print("Converting image to complex dtype...")
    image_t = image_t.to(torch.complex128)  # Convert to complex type

# Now pass the image to the nufft_ob function
kdata = nufft_ob(image_t, ktraj)

# add some noise (robustness test)
#siglevel = torch.abs(kdata).mean()
#kdata = kdata + (siglevel/5) * torch.randn(kdata.shape).to(kdata)
print(kdata.shape)

In [ ]:
# plot the k-space data on log-scale
kdata_numpy = np.reshape(kdata.cpu().numpy(), (nspokes,num_samples_per_ray))
plt.imshow(np.log10(np.absolute(kdata_numpy)))
plt.gray()
plt.title('k-space data, log10 scale')
plt.show()

In [ ]:
print(ktraj.shape)
# adjnufft back
# method 1: no density compensation (blurry image)
image_blurry = adjnufft_ob(kdata, ktraj)

# method 2: use density compensation
dcomp = tkbn.calc_density_compensation_function(ktraj=ktraj, im_size=im_size)
image_sharp = adjnufft_ob(kdata * dcomp, ktraj)

In [ ]:
# show the images
image_blurry_numpy = np.squeeze(image_blurry.cpu().numpy())

image_sharp_numpy = np.squeeze(image_sharp.cpu().numpy())

print(image_sharp_numpy.shape)
print(image.shape)

diff_image = np.abs(image_sharp - image)
diff_image = np.squeeze(diff_image)

plt.figure(0)
plt.imshow(np.absolute(image_blurry_numpy))
plt.gray()
plt.colorbar()
plt.title('blurry image')

plt.figure(1)
plt.imshow(np.absolute(image_sharp_numpy))
plt.gray()
plt.colorbar()
plt.title('sharp image (with Pipe dcomp)')

plt.figure(2)
plt.imshow(np.absolute(image))
plt.gray()
plt.colorbar()
plt.title('Original Image')

plt.figure(3)
plt.imshow(diff_image, cmap='hot')
plt.colorbar()
plt.title('Difference Image')

plt.show()
